In [1]:
import sys
sys.path.append('/Users/liangwang/GitHub/DataAnalysis')

In [2]:
import numpy as np
import sys, os
from pathlib import Path
import pandas as pd
import copy
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
from plotly.colors import n_colors
import math
import datetime as dt
from sklearn.metrics import r2_score
import plotly.colors
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from scipy.stats import mannwhitneyu
from scipy.interpolate import griddata
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
import scipy
import importlib
import pickle
import psutil 
import gc

# ----- Internal Dependencies -------#
nb_dir = os.path.split(os.getcwd())[0]
sys.path.append("../..")
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
from General import FileRead
from General import AccuracyMetrics
from General.GeneralFunctions import get_add_to_dict
from General import TableManipulations
from FeatureGeneration import GeneralFeatures
from General import FileWrite

from Plot.PdfHelper import PdfHelper
from Plot.PlotMaker import PlotMaker
# from Plot import Voltage_Plots
from Plot import PlotFunctions
# from Plot import AdHocPlots
# from Plot import Sim_Plots

# widget and notebook stuff
from ipywidgets import interact, fixed
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook"

load data

In [3]:
training_group = ['eBlinq17', 'eBlinq19a','eBlinq19b','eBlinq19c','eBlinq20','eBlinq22','eBlinq23','eBlinq25','eBlinqRingOverlay&NiMNA','iBlinqAcet','iBlinqFW231','iBlinqOverlay','Pre-Piv-2-training']
# test_group = ['Pre-Piv-2-training']
# training_group = ['eBlinq19c','eBlinq20','eBlinq22','eBlinq23','eBlinq25','eBlinqRingOverlay&NiMNA','iBlinqFW231','iBlinqOverlay']
# test_group = ['Pre-Piv-2-training','eBlinqMayStudy']
# test_group = ['CPP-008-27', 'CPP-008-25-Validation']
test_group = ['CPP-008-27']
training_time_window = [0.083,5]
all_training_channel = ['1','3','4']
# training_chemistry = '2.6.11'
# test_chemistry = '2.6.11'
iso_thresh = 25
feature_hour_window = '8hr'
feature_col = [
    'time_from_start_featureGeneration',
    f'ewm_average_{feature_hour_window}_iir_filt_model_output_s1_glucoseFeatureGeneration',
    f'ewm_average_{feature_hour_window}_iir_filt_model_output_s3_glucoseFeatureGeneration',
    f'ewm_average_{feature_hour_window}_iir_filt_model_output_s4_glucoseFeatureGeneration',
    f'estimate_standard_deviation_{feature_hour_window}_iir_filt_model_output_s1_glucoseFeatureGeneration',
    f'estimate_standard_deviation_{feature_hour_window}_iir_filt_model_output_s3_glucoseFeatureGeneration',
    f'estimate_standard_deviation_{feature_hour_window}_iir_filt_model_output_s4_glucoseFeatureGeneration',
]
gluc_col = [
    'model_output_s1_calculateGlucose','model_output_s3_calculateGlucose','model_output_s4_calculateGlucose',
    'biolinq_s1_channelFaultDetection','biolinq_s3_channelFaultDetection','biolinq_s4_channelFaultDetection',
]

gluc_ref_col = [
    'Study','Sensor_Id',
    'retro_model_output_s1_calculateGlucose','retro_model_output_s3_calculateGlucose','retro_model_output_s4_calculateGlucose','cgm_adj','ysi','acck',
    'retro_rmse_s1','retro_rmse_s3','retro_rmse_s4'
]

ground_truth_col = [
    'retro_slope_s1','retro_intercept_s1',
    'retro_slope_s3','retro_intercept_s3',
    'retro_slope_s4','retro_intercept_s4',
]
all_cols = feature_col + gluc_col + gluc_ref_col + ground_truth_col
percentile_lb = 0.01
percentile_ub = 0.99
slope_lb = 0.1
slope_ub = 10
intercept_lb = -100
intercept_ub = 100
rmse_quantile_thresh = 1

In [4]:
psutil.virtual_memory().available * 100 / psutil.virtual_memory().total

72.2682237625122

In [5]:
df_orig_recent=pd.read_csv('/Users/liangwang/Library/CloudStorage/OneDrive-Biolinq/Gen 1/Algorithm Development/Gen 1 Model Optimization/20240521/agg_mfg_ref_sample_df_from17.csv',usecols=all_cols)
df_orig_may=pd.read_csv('/Users/liangwang/Library/CloudStorage/OneDrive-Biolinq/Gen 1/Algorithm Development/Gen 1 Model Optimization/agg_sample_df_ref_May_Study.csv',usecols=all_cols)

In [6]:
psutil.virtual_memory().available * 100 / psutil.virtual_memory().total

54.85396385192871

In [7]:
df_orig = pd.concat([df_orig_recent, df_orig_may], join="inner")

In [8]:
df_orig_recent.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14308248 entries, 0 to 14308247
Data columns (total 30 columns):
 #   Column                                                                             Dtype  
---  ------                                                                             -----  
 0   Sensor_Id                                                                          object 
 1   Study                                                                              object 
 2   acck                                                                               float64
 3   biolinq_s1_channelFaultDetection                                                   float64
 4   biolinq_s3_channelFaultDetection                                                   float64
 5   biolinq_s4_channelFaultDetection                                                   float64
 6   cgm_adj                                                                            float64
 7   estimate_standar

In [9]:
df_orig_may.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1727833 entries, 0 to 1727832
Data columns (total 30 columns):
 #   Column                                                                             Dtype  
---  ------                                                                             -----  
 0   Sensor_Id                                                                          object 
 1   Study                                                                              object 
 2   acck                                                                               float64
 3   biolinq_s1_channelFaultDetection                                                   float64
 4   biolinq_s3_channelFaultDetection                                                   float64
 5   biolinq_s4_channelFaultDetection                                                   float64
 6   cgm_adj                                                                            float64
 7   estimate_standard_

In [10]:
df_orig.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16036081 entries, 0 to 1727832
Data columns (total 30 columns):
 #   Column                                                                             Dtype  
---  ------                                                                             -----  
 0   Sensor_Id                                                                          object 
 1   Study                                                                              object 
 2   acck                                                                               float64
 3   biolinq_s1_channelFaultDetection                                                   float64
 4   biolinq_s3_channelFaultDetection                                                   float64
 5   biolinq_s4_channelFaultDetection                                                   float64
 6   cgm_adj                                                                            float64
 7   estimate_standard

In [11]:
psutil.virtual_memory().available * 100 / psutil.virtual_memory().total

49.0464448928833

In [12]:
del df_orig_recent
del df_orig_may

In [13]:
psutil.virtual_memory().available * 100 / psutil.virtual_memory().total

53.87907028198242

In [14]:
gc.collect()

89

In [15]:
psutil.virtual_memory().available * 100 / psutil.virtual_memory().total

53.87997627258301

In [16]:
df_orig['Study'].unique()

array(['eBlinq17', 'eBlinq19a', 'eBlinq19b', 'eBlinq19c', 'eBlinq20',
       'eBlinqRingOverlay&NiMNA', 'eBlinq22', 'eBlinq23',
       'Pre-Piv-2-training', 'eBlinq25', 'iBlinqAcet', 'iBlinqFW231',
       'iBlinqOverlay', 'CPP-008-27', 'CPP-008-25-Validation'],
      dtype=object)

Setting

In [17]:
df = df_orig[all_cols].copy()
feature_name_avg_ch1 = f'ewm_average_{feature_hour_window}_iir_filt_model_output_s1_glucoseFeatureGeneration'
feature_name_avg_ch1_lb = df[feature_name_avg_ch1].quantile(q=percentile_lb)
feature_name_avg_ch1_ub = df[feature_name_avg_ch1].quantile(q=percentile_ub)
feature_name_std_ch1 = f'estimate_standard_deviation_{feature_hour_window}_iir_filt_model_output_s1_glucoseFeatureGeneration'
feature_name_std_ch1_lb = df[feature_name_std_ch1].quantile(q=percentile_lb)
feature_name_std_ch1_ub = df[feature_name_std_ch1].quantile(q=percentile_ub)
feature_name_avg_ch3 = f'ewm_average_{feature_hour_window}_iir_filt_model_output_s3_glucoseFeatureGeneration'
feature_name_avg_ch3_lb = df[feature_name_avg_ch3].quantile(q=percentile_lb)
feature_name_avg_ch3_ub = df[feature_name_avg_ch3].quantile(q=percentile_ub)
feature_name_std_ch3 = f'estimate_standard_deviation_{feature_hour_window}_iir_filt_model_output_s3_glucoseFeatureGeneration'
feature_name_std_ch3_lb = df[feature_name_std_ch3].quantile(q=percentile_lb)
feature_name_std_ch3_ub = df[feature_name_std_ch3].quantile(q=percentile_ub)
feature_name_avg_ch4 = f'ewm_average_{feature_hour_window}_iir_filt_model_output_s4_glucoseFeatureGeneration'
feature_name_avg_ch4_lb = df[feature_name_avg_ch4].quantile(q=percentile_lb)
feature_name_avg_ch4_ub = df[feature_name_avg_ch4].quantile(q=percentile_ub)
feature_name_std_ch4 = f'estimate_standard_deviation_{feature_hour_window}_iir_filt_model_output_s4_glucoseFeatureGeneration'
feature_name_std_ch4_lb = df[feature_name_std_ch4].quantile(q=percentile_lb)
feature_name_std_ch4_ub = df[feature_name_std_ch4].quantile(q=percentile_ub)

rmse_s1_thresh = df['retro_rmse_s1'].quantile(q=rmse_quantile_thresh)
rmse_s3_thresh = df['retro_rmse_s3'].quantile(q=rmse_quantile_thresh)
rmse_s4_thresh = df['retro_rmse_s4'].quantile(q=rmse_quantile_thresh)

training_index = df['Study'].isin(training_group) & df['time_from_start_featureGeneration'].between(training_time_window[0],training_time_window[1])
df_training = df.loc[training_index,:].dropna(subset = feature_col + ground_truth_col).reset_index(drop=True)
valid_training_index = df['Study'].isin(training_group) & df['time_from_start_featureGeneration'].between(training_time_window[0],training_time_window[1]) & \
                        df[feature_name_avg_ch1].between(feature_name_avg_ch1_lb,feature_name_avg_ch1_ub) & \
                        df[feature_name_std_ch1].between(feature_name_std_ch1_lb,feature_name_std_ch1_ub) & \
                        df[feature_name_avg_ch3].between(feature_name_avg_ch3_lb,feature_name_avg_ch3_ub) & \
                        df[feature_name_std_ch3].between(feature_name_std_ch3_lb,feature_name_std_ch3_ub) & \
                        df[feature_name_avg_ch4].between(feature_name_avg_ch4_lb,feature_name_avg_ch4_ub) & \
                        df[feature_name_std_ch4].between(feature_name_std_ch4_lb,feature_name_std_ch4_ub) & \
                        df['retro_slope_s1'].between(slope_lb,slope_ub) & df['retro_intercept_s1'].between(intercept_lb,intercept_ub) & \
                        df['retro_slope_s3'].between(slope_lb,slope_ub) & df['retro_intercept_s3'].between(intercept_lb,intercept_ub) & \
                        df['retro_slope_s4'].between(slope_lb,slope_ub) & df['retro_intercept_s4'].between(intercept_lb,intercept_ub) & \
                        df['retro_rmse_s1'].le(rmse_s1_thresh) & df['retro_rmse_s3'].le(rmse_s3_thresh) & df['retro_rmse_s4'].le(rmse_s4_thresh)
df_training_valid = df.loc[valid_training_index,:].dropna(subset = feature_col + ground_truth_col).reset_index(drop=True)
print(f'Training data has shape {df_training.shape}')
print(f'Training data has shape {df_training_valid.shape}')
good_data_ratio = len(df_training_valid)/len(df_training)
print(f'Ratio of training good data is {good_data_ratio}')
x_train = df_training_valid[feature_col]
y_train = df_training_valid[ground_truth_col].to_numpy()
min_max_scaler = sklearn.preprocessing.MinMaxScaler()
x_train_norm = min_max_scaler.fit_transform(x_train)

test_index = df['Study'].isin(test_group) & df['time_from_start_featureGeneration'].between(training_time_window[0],training_time_window[1])
df_test = df.loc[test_index,:].dropna(subset = feature_col + ground_truth_col).reset_index(drop=True)
valid_test_index = df['Study'].isin(test_group) & df['time_from_start_featureGeneration'].between(training_time_window[0],training_time_window[1]) & \
                        df[feature_name_avg_ch1].between(feature_name_avg_ch1_lb,feature_name_avg_ch1_ub) & \
                        df[feature_name_std_ch1].between(feature_name_std_ch1_lb,feature_name_std_ch1_ub) & \
                        df[feature_name_avg_ch3].between(feature_name_avg_ch3_lb,feature_name_avg_ch3_ub) & \
                        df[feature_name_std_ch3].between(feature_name_std_ch3_lb,feature_name_std_ch3_ub) & \
                        df[feature_name_avg_ch4].between(feature_name_avg_ch4_lb,feature_name_avg_ch4_ub) & \
                        df[feature_name_std_ch4].between(feature_name_std_ch4_lb,feature_name_std_ch4_ub) & \
                        df['retro_slope_s1'].between(slope_lb,slope_ub) & df['retro_intercept_s1'].between(intercept_lb,intercept_ub) & \
                        df['retro_slope_s3'].between(slope_lb,slope_ub) & df['retro_intercept_s3'].between(intercept_lb,intercept_ub) & \
                        df['retro_slope_s4'].between(slope_lb,slope_ub) & df['retro_intercept_s4'].between(intercept_lb,intercept_ub) & \
                        df['retro_rmse_s1'].le(rmse_s1_thresh) & df['retro_rmse_s3'].le(rmse_s3_thresh) & df['retro_rmse_s4'].le(rmse_s4_thresh)
df_test_valid = df.loc[valid_test_index,:].dropna(subset = feature_col + ground_truth_col).reset_index(drop=True)
print(f'Test data has shape {df_test.shape}')
print(f'Test data has shape {df_test_valid.shape}')
good_test_data_ratio = len(df_test_valid)/len(df_test)
print(f'Ratio of good test data is {good_test_data_ratio}')
x_test = df_test_valid[feature_col]
x_test_norm = min_max_scaler.transform(x_test)
y_test = df_test_valid[ground_truth_col].to_numpy()

Training data has shape (10517625, 30)
Training data has shape (9010156, 30)
Ratio of training good data is 0.856672109910745
Test data has shape (800745, 30)
Test data has shape (700990, 30)
Ratio of good test data is 0.8754222630175649


In [18]:
self_cal_model_dict = {}

In [19]:
# For channel 1 slope
i = 0
print(f'Optimizing parameter {i}')
self_cal_model = MLPRegressor(hidden_layer_sizes=(10,10,10,1),activation='logistic',solver='adam',alpha=0.0001,max_iter=int(1e4),tol=1e-4,learning_rate='adaptive',random_state=20140806)
self_cal_model.fit(x_train_norm,y_train[:,i])
predict_slope = self_cal_model.predict(x_train_norm)
print(f'Max of predict_slope is {predict_slope.max()}')
print(f'Min of predict_slope is {predict_slope.min()}')
# predict_slope[predict_slope<0.12] = 1
# predict_slope[predict_slope>2.4] = 1
mse_train_selfcal = np.mean((predict_slope - y_train[:,i])**2)
mse_train_today = np.mean((1-y_train[:,i])**2)
print(f'The model today has MSE={mse_train_today}')
print(f'The self cal model has MSE={mse_train_selfcal}')

predict_slope_test = self_cal_model.predict(x_test_norm)
print(f'Max of predict_slope_test is {predict_slope_test.max()}')
print(f'Min of predict_slope_test is {predict_slope_test.min()}')
# predict_slope_test[predict_slope_test<0.12] = 1
# predict_slope_test[predict_slope_test>2.4] = 1    
mse_test_selfcal = np.mean((predict_slope_test - y_test[:,i])**2)
mse_test_today = np.mean((1-y_test[:,i])**2)
print(f'The model today on test has MSE={mse_test_today}')
print(f'The self cal model on test has MSE={mse_test_selfcal}')
self_cal_model_dict[f'self_cal_model_{i}'] = self_cal_model

Optimizing parameter 0
Max of predict_slope is 1.5097663252724818
Min of predict_slope is 0.5411007482493388
The model today has MSE=0.10326341837504217
The self cal model has MSE=0.07337429382960446
Max of predict_slope_test is 1.5109781948176775
Min of predict_slope_test is 0.5432897856363051
The model today on test has MSE=0.08981732119579276
The self cal model on test has MSE=0.1109455344909849


In [20]:
# For channel 1 intercept
i = 1
print(f'Optimizing parameter {i}')
self_cal_model = MLPRegressor(hidden_layer_sizes=(10,10,10,1),activation='logistic',solver='adam',alpha=1e-4,max_iter=int(1e4),tol=1e-4,learning_rate='adaptive',random_state=20140806)
self_cal_model.fit(x_train_norm,y_train[:,i])
predict_intercept = self_cal_model.predict(x_train_norm)
print(f'Max of predict_intercept is {predict_intercept.max()}')
print(f'Min of predict_intercept is {predict_intercept.min()}')    
# predict_intercept[predict_intercept<-100] = 0
# predict_intercept[predict_intercept>100] = 0   
mse_train_selfcal = np.mean((predict_intercept - y_train[:,i])**2)
mse_train_today = np.mean(y_train[:,i]**2)
print(f'The model today has MSE={mse_train_today}')
print(f'The self cal model has MSE={mse_train_selfcal}')

predict_intercept_test = self_cal_model.predict(x_test_norm)
print(f'Max of predict_intercept_test is {predict_intercept_test.max()}')
print(f'Min of predict_intercept_test is {predict_intercept_test.min()}')    
# predict_intercept_test[predict_intercept_test<-100] = 0
# predict_intercept_test[predict_intercept_test>100] = 0     
mse_test_selfcal = np.mean((predict_intercept_test - y_test[:,i])**2)
mse_test_today = np.mean(y_test[:,i]**2)
print(f'The model today on test has MSE={mse_test_today}')
print(f'The self cal model on test has MSE={mse_test_selfcal}')
self_cal_model_dict[f'self_cal_model_{i}'] = self_cal_model

Optimizing parameter 1
Max of predict_intercept is 27.35787374904198
Min of predict_intercept is -106.83652246385364
The model today has MSE=897.27845397517
The self cal model has MSE=695.6429929880377
Max of predict_intercept_test is 27.35857496084264
Min of predict_intercept_test is -106.83756761899988
The model today on test has MSE=448.7359626373067
The self cal model on test has MSE=981.0808875909207


In [21]:
ch1_slope_train = self_cal_model_dict['self_cal_model_0'].predict(x_train_norm)
print(f'Max of predict_slope is {ch1_slope_train.max()}')
print(f'Min of predict_slope is {ch1_slope_train.min()}')
ch1_intercept_train = self_cal_model_dict['self_cal_model_1'].predict(x_train_norm)
print(f'Max of predict_intercept is {ch1_intercept_train.max()}')
print(f'Min of predict_intercept is {ch1_intercept_train.min()}')
df_training_valid['model_output_s1_selfcal'] = df_training_valid['model_output_s1_calculateGlucose'].sub(ch1_intercept_train).div(ch1_slope_train)
n_ch1_train_today = df_training_valid['model_output_s1_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).count()
mrd_ch1_train_today = df_training_valid['model_output_s1_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).mean()
stdrd_ch1_train_today = df_training_valid['model_output_s1_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).std()
mard_ch1_train_today = df_training_valid['model_output_s1_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).abs().mean()
n_ch1_train_selfcal = df_training_valid['model_output_s1_selfcal'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).count()
mrd_ch1_train_selfcal = df_training_valid['model_output_s1_selfcal'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).mean()
stdrd_ch1_train_selfcal = df_training_valid['model_output_s1_selfcal'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).std()
mard_ch1_train_selfcal = df_training_valid['model_output_s1_selfcal'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).abs().mean()
n_ch1_train_retro = df_training_valid['retro_model_output_s1_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).count()
mrd_ch1_train_retro = df_training_valid['retro_model_output_s1_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).mean()
stdrd_ch1_train_retro = df_training_valid['retro_model_output_s1_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).std()
mard_ch1_train_retro = df_training_valid['retro_model_output_s1_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).abs().mean()
print(f'Training data for ch1 glucose by model today N = {n_ch1_train_today}, MRD = {mrd_ch1_train_today}, StdRD = {stdrd_ch1_train_today}, MARD = {mard_ch1_train_today}')
print(f'Training data for ch1 glucose by selfcal N = {n_ch1_train_selfcal}, MRD = {mrd_ch1_train_selfcal}, StdRD = {stdrd_ch1_train_selfcal}, MARD = {mard_ch1_train_selfcal}')
print(f'Training data for ch1 glucose by retro N = {n_ch1_train_retro}, MRD = {mrd_ch1_train_retro}, StdRD = {stdrd_ch1_train_retro}, MARD = {mard_ch1_train_retro}')

ch1_slope_test = self_cal_model_dict['self_cal_model_0'].predict(x_test_norm)
print(f'Max of predict_slope is {ch1_slope_test.max()}')
print(f'Min of predict_slope is {ch1_slope_test.min()}')
ch1_intercept_test = self_cal_model_dict['self_cal_model_1'].predict(x_test_norm)
print(f'Max of predict_intercept is {ch1_intercept_test.max()}')
print(f'Min of predict_intercept is {ch1_intercept_test.min()}')
df_test_valid['model_output_s1_selfcal'] = df_test_valid['model_output_s1_calculateGlucose'].sub(ch1_intercept_test).div(ch1_slope_test)
n_ch1_test_today = df_test_valid['model_output_s1_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).count()
mrd_ch1_test_today = df_test_valid['model_output_s1_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).mean()
stdrd_ch1_test_today = df_test_valid['model_output_s1_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).std()
mard_ch1_test_today = df_test_valid['model_output_s1_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).abs().mean()
n_ch1_test_selfcal = df_test_valid['model_output_s1_selfcal'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).count()
mrd_ch1_test_selfcal = df_test_valid['model_output_s1_selfcal'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).mean()
stdrd_ch1_test_selfcal = df_test_valid['model_output_s1_selfcal'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).std()
mard_ch1_test_selfcal = df_test_valid['model_output_s1_selfcal'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).abs().mean()
n_ch1_test_retro = df_test_valid['retro_model_output_s1_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).count()
mrd_ch1_test_retro = df_test_valid['retro_model_output_s1_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).mean()
stdrd_ch1_test_retro = df_test_valid['retro_model_output_s1_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).std()
mard_ch1_test_retro = df_test_valid['retro_model_output_s1_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).abs().mean()
print(f'Test data for ch1 glucose by model today N = {n_ch1_test_today}, MRD = {mrd_ch1_test_today}, StdRD = {stdrd_ch1_test_today}, MARD = {mard_ch1_test_today}')
print(f'Test data for ch1 glucose by selfcal N = {n_ch1_test_selfcal}, MRD = {mrd_ch1_test_selfcal}, StdRD = {stdrd_ch1_test_selfcal}, MARD = {mard_ch1_test_selfcal}')
print(f'Test data for ch1 glucose by retro N = {n_ch1_test_retro}, MRD = {mrd_ch1_test_retro}, StdRD = {stdrd_ch1_test_retro}, MARD = {mard_ch1_test_retro}')

Max of predict_slope is 1.5097663252724818
Min of predict_slope is 0.5411007482493388
Max of predict_intercept is 27.35787374904198
Min of predict_intercept is -106.83652246385364
Training data for ch1 glucose by model today N = 6225452, MRD = -0.05327920529170002, StdRD = 0.3409502380123258, MARD = 0.24485890017191944
Training data for ch1 glucose by selfcal N = 6225452, MRD = 0.015263685588758673, StdRD = 0.32553903424206404, MARD = 0.22127135729497097
Training data for ch1 glucose by retro N = 6225452, MRD = -0.00018597001299262042, StdRD = 0.3397448621300012, MARD = 0.1907338361626772
Max of predict_slope is 1.5109781948176775
Min of predict_slope is 0.5432897856363051
Max of predict_intercept is 27.35857496084264
Min of predict_intercept is -106.83756761899988
Test data for ch1 glucose by model today N = 659457, MRD = -0.217432380512837, StdRD = 0.23791465623552785, MARD = 0.263162867835626
Test data for ch1 glucose by selfcal N = 659457, MRD = -0.139917384496509, StdRD = 0.304627

In [22]:
df = df_orig[all_cols].copy()

df_train_ysi = df.loc[training_index]
x_train_ysi = df_train_ysi[feature_col]
x_train_ysi_norm = min_max_scaler.transform(x_train_ysi)
ch1_slope_train = self_cal_model_dict['self_cal_model_0'].predict(x_train_ysi_norm)
print(f'Max of predict_slope is {ch1_slope_train.max()}')
print(f'Min of predict_slope is {ch1_slope_train.min()}')
ch1_intercept_train = self_cal_model_dict['self_cal_model_1'].predict(x_train_ysi_norm)
print(f'Max of predict_intercept is {ch1_intercept_train.max()}')
print(f'Min of predict_intercept is {ch1_intercept_train.min()}')
df_train_ysi['model_output_s1_selfcal'] = df_train_ysi['model_output_s1_calculateGlucose'].sub(ch1_intercept_train).div(ch1_slope_train)
n_ch1_train_today = df_train_ysi['model_output_s1_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).count()
mrd_ch1_train_today = df_train_ysi['model_output_s1_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).mean()
stdrd_ch1_train_today = df_train_ysi['model_output_s1_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).std()
mard_ch1_train_today = df_train_ysi['model_output_s1_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).abs().mean()
n_ch1_train_selfcal = df_train_ysi['model_output_s1_selfcal'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).count()
mrd_ch1_train_selfcal = df_train_ysi['model_output_s1_selfcal'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).mean()
stdrd_ch1_train_selfcal = df_train_ysi['model_output_s1_selfcal'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).std()
mard_ch1_train_selfcal = df_train_ysi['model_output_s1_selfcal'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).abs().mean()
n_ch1_train_retro = df_train_ysi['retro_model_output_s1_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).count()
mrd_ch1_train_retro = df_train_ysi['retro_model_output_s1_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).mean()
stdrd_ch1_train_retro = df_train_ysi['retro_model_output_s1_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).std()
mard_ch1_train_retro = df_train_ysi['retro_model_output_s1_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).abs().mean()
print(f'Training data for ch1 glucose by model today N = {n_ch1_train_today}, MRD = {mrd_ch1_train_today}, StdRD = {stdrd_ch1_train_today}, MARD = {mard_ch1_train_today}')
print(f'Training data for ch1 glucose by selfcal N = {n_ch1_train_selfcal}, MRD = {mrd_ch1_train_selfcal}, StdRD = {stdrd_ch1_train_selfcal}, MARD = {mard_ch1_train_selfcal}')
print(f'Training data for ch1 glucose by retro N = {n_ch1_train_retro}, MRD = {mrd_ch1_train_retro}, StdRD = {stdrd_ch1_train_retro}, MARD = {mard_ch1_train_retro}')

df_test_ysi = df.loc[test_index]
x_test_ysi = df_test_ysi[feature_col]
x_test_ysi_norm = min_max_scaler.transform(x_test_ysi)
ch1_slope_test = self_cal_model_dict['self_cal_model_0'].predict(x_test_ysi_norm)
print(f'Max of predict_slope is {ch1_slope_test.max()}')
print(f'Min of predict_slope is {ch1_slope_test.min()}')
ch1_intercept_test = self_cal_model_dict['self_cal_model_1'].predict(x_test_ysi_norm)
print(f'Max of predict_intercept is {ch1_intercept_test.max()}')
print(f'Min of predict_intercept is {ch1_intercept_test.min()}')
df_test_ysi['model_output_s1_selfcal'] = df_test_ysi['model_output_s1_calculateGlucose'].sub(ch1_intercept_test).div(ch1_slope_test)
n_ch1_test_today = df_test_ysi['model_output_s1_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch1_test_today = df_test_ysi['model_output_s1_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch1_test_today = df_test_ysi['model_output_s1_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch1_test_today = df_test_ysi['model_output_s1_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
n_ch1_test_selfcal = df_test_ysi['model_output_s1_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch1_test_selfcal = df_test_ysi['model_output_s1_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch1_test_selfcal = df_test_ysi['model_output_s1_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch1_test_selfcal = df_test_ysi['model_output_s1_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
n_ch1_test_retro = df_test_ysi['retro_model_output_s1_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch1_test_retro = df_test_ysi['retro_model_output_s1_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch1_test_retro = df_test_ysi['retro_model_output_s1_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch1_test_retro = df_test_ysi['retro_model_output_s1_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
print(f'Test data for ch1 glucose by model today N = {n_ch1_test_today}, MRD = {mrd_ch1_test_today}, StdRD = {stdrd_ch1_test_today}, MARD = {mard_ch1_test_today}')
print(f'Test data for ch1 glucose by selfcal N = {n_ch1_test_selfcal}, MRD = {mrd_ch1_test_selfcal}, StdRD = {stdrd_ch1_test_selfcal}, MARD = {mard_ch1_test_selfcal}')
print(f'Test data for ch1 glucose by retro N = {n_ch1_test_retro}, MRD = {mrd_ch1_test_retro}, StdRD = {stdrd_ch1_test_retro}, MARD = {mard_ch1_test_retro}')

Max of predict_slope is 1.5178701998508712
Min of predict_slope is 0.5410513277164305
Max of predict_intercept is 27.35787374904198
Min of predict_intercept is -106.83652246385364


/var/folders/nr/d5llxsx51b3fljj36q8cwm1c0000gp/T/ipykernel_8323/2400631413.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Training data for ch1 glucose by model today N = 328214, MRD = -0.03899769274765646, StdRD = 0.308170418856884, MARD = 0.2259532496283006
Training data for ch1 glucose by selfcal N = 328214, MRD = 0.012058417904563946, StdRD = 0.2945880888770862, MARD = 0.2142369005473645
Training data for ch1 glucose by retro N = 323773, MRD = 0.03223530188190728, StdRD = 0.4318273886857706, MARD = 0.19402882282784342
Max of predict_slope is 1.5137007272140575
Min of predict_slope is 0.541057089222587
Max of predict_intercept is 27.35857496084264
Min of predict_intercept is -106.83756761899988
Test data for ch1 glucose by model today N = 44042, MRD = -0.22569659157780864, StdRD = 0.21951972992760932, MARD = 0.25244884098748405
Test data for ch1 glucose by selfcal N = 44042, MRD = -0.016016892132042257, StdRD = 0.5892412473025731, MARD = 0.4004275318112295
Test data for ch1 glucose by retro N = 44042, MRD = 0.002304639647720784, StdRD = 0.25013297878584756, MARD = 0.17623555066067595


/var/folders/nr/d5llxsx51b3fljj36q8cwm1c0000gp/T/ipykernel_8323/2400631413.py:38: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [23]:
df = df_orig[all_cols].copy()

df_test_ysi = df.loc[training_index]
x_train_ysi = df_test_ysi[feature_col]
x_train_ysi_norm = min_max_scaler.transform(x_train_ysi)

ch1_slope_train = self_cal_model_dict['self_cal_model_0'].predict(x_train_ysi_norm)
print(f'Max of predict_slope is {ch1_slope_train.max()}')
print(f'Min of predict_slope is {ch1_slope_train.min()}')
ch1_intercept_train = self_cal_model_dict['self_cal_model_1'].predict(x_train_ysi_norm)
print(f'Max of predict_intercept is {ch1_intercept_train.max()}')
print(f'Min of predict_intercept is {ch1_intercept_train.min()}')
df_test_ysi['biolinq_s1_selfcal'] = df_test_ysi['biolinq_s1_channelFaultDetection'].sub(ch1_intercept_train).div(ch1_slope_train)
n_ch1_train_today = df_test_ysi['biolinq_s1_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch1_train_today = df_test_ysi['biolinq_s1_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch1_train_today = df_test_ysi['biolinq_s1_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch1_train_today = df_test_ysi['biolinq_s1_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
n_ch1_train_selfcal = df_test_ysi['biolinq_s1_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch1_train_selfcal = df_test_ysi['biolinq_s1_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch1_train_selfcal = df_test_ysi['biolinq_s1_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch1_train_selfcal = df_test_ysi['biolinq_s1_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
n_ch1_train_retro = df_test_ysi['retro_model_output_s1_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch1_train_retro = df_test_ysi['retro_model_output_s1_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch1_train_retro = df_test_ysi['retro_model_output_s1_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch1_train_retro = df_test_ysi['retro_model_output_s1_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
print(f'Training data for ch1 glucose by model today N = {n_ch1_train_today}, MRD = {mrd_ch1_train_today}, StdRD = {stdrd_ch1_train_today}, MARD = {mard_ch1_train_today}')
print(f'Training data for ch1 glucose by selfcal N = {n_ch1_train_selfcal}, MRD = {mrd_ch1_train_selfcal}, StdRD = {stdrd_ch1_train_selfcal}, MARD = {mard_ch1_train_selfcal}')
print(f'Training data for ch1 glucose by retro N = {n_ch1_train_retro}, MRD = {mrd_ch1_train_retro}, StdRD = {stdrd_ch1_train_retro}, MARD = {mard_ch1_train_retro}')

df_test_ysi = df.loc[test_index]
x_test_ysi = df_test_ysi[feature_col]
x_test_ysi_norm = min_max_scaler.transform(x_test_ysi)
ch1_slope_test = self_cal_model_dict['self_cal_model_0'].predict(x_test_ysi_norm)
print(f'Max of predict_slope is {ch1_slope_test.max()}')
print(f'Min of predict_slope is {ch1_slope_test.min()}')
ch1_intercept_test = self_cal_model_dict['self_cal_model_1'].predict(x_test_ysi_norm)
print(f'Max of predict_intercept is {ch1_intercept_test.max()}')
print(f'Min of predict_intercept is {ch1_intercept_test.min()}')
df_test_ysi['biolinq_s1_selfcal'] = df_test_ysi['biolinq_s1_channelFaultDetection'].sub(ch1_intercept_test).div(ch1_slope_test)
n_ch1_test_today = df_test_ysi['biolinq_s1_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch1_test_today = df_test_ysi['biolinq_s1_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch1_test_today = df_test_ysi['biolinq_s1_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch1_test_today = df_test_ysi['biolinq_s1_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
n_ch1_test_selfcal = df_test_ysi['biolinq_s1_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch1_test_selfcal = df_test_ysi['biolinq_s1_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch1_test_selfcal = df_test_ysi['biolinq_s1_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch1_test_selfcal = df_test_ysi['biolinq_s1_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
n_ch1_test_retro = df_test_ysi['retro_model_output_s1_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch1_test_retro = df_test_ysi['retro_model_output_s1_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch1_test_retro = df_test_ysi['retro_model_output_s1_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch1_test_retro = df_test_ysi['retro_model_output_s1_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
print(f'Test data for ch1 glucose by model today N = {n_ch1_test_today}, MRD = {mrd_ch1_test_today}, StdRD = {stdrd_ch1_test_today}, MARD = {mard_ch1_test_today}')
print(f'Test data for ch1 glucose by selfcal N = {n_ch1_test_selfcal}, MRD = {mrd_ch1_test_selfcal}, StdRD = {stdrd_ch1_test_selfcal}, MARD = {mard_ch1_test_selfcal}')
print(f'Test data for ch1 glucose by retro N = {n_ch1_test_retro}, MRD = {mrd_ch1_test_retro}, StdRD = {stdrd_ch1_test_retro}, MARD = {mard_ch1_test_retro}')

Max of predict_slope is 1.5178701998508712
Min of predict_slope is 0.5410513277164305
Max of predict_intercept is 27.35787374904198
Min of predict_intercept is -106.83652246385364


/var/folders/nr/d5llxsx51b3fljj36q8cwm1c0000gp/T/ipykernel_8323/1501457173.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Training data for ch1 glucose by model today N = 286796, MRD = 0.013744876961750734, StdRD = 0.23517822865477775, MARD = 0.17880000000212173
Training data for ch1 glucose by selfcal N = 286796, MRD = 0.05940115725619481, StdRD = 0.23489810026009378, MARD = 0.17981880605534284
Training data for ch1 glucose by retro N = 323773, MRD = 0.03223530188190728, StdRD = 0.4318273886857706, MARD = 0.19402882282784342
Max of predict_slope is 1.5137007272140575
Min of predict_slope is 0.541057089222587
Max of predict_intercept is 27.35857496084264
Min of predict_intercept is -106.83756761899988
Test data for ch1 glucose by model today N = 39614, MRD = -0.19201181631131736, StdRD = 0.18187142372651005, MARD = 0.2216389180078642
Test data for ch1 glucose by selfcal N = 39614, MRD = 0.014463087220663653, StdRD = 0.568821867700014, MARD = 0.3765788293763973
Test data for ch1 glucose by retro N = 44042, MRD = 0.002304639647720784, StdRD = 0.25013297878584756, MARD = 0.17623555066067595


/var/folders/nr/d5llxsx51b3fljj36q8cwm1c0000gp/T/ipykernel_8323/1501457173.py:39: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [24]:
[ch1_slope_train.min(),
ch1_slope_train.max(),
ch1_intercept_train.min(),
ch1_intercept_train.max(),
ch1_slope_test.min(),
ch1_slope_test.max(),
ch1_intercept_test.min(),
ch1_intercept_test.max()]

[0.5410513277164305,
 1.5178701998508712,
 -106.83652246385364,
 27.35787374904198,
 0.541057089222587,
 1.5137007272140575,
 -106.83756761899988,
 27.35857496084264]

In [37]:
# For channel 3 slope
i = 2
print(f'Optimizing parameter {i}')
self_cal_model = MLPRegressor(hidden_layer_sizes=(10,10,10,1),activation='logistic',solver='adam',alpha=1e-4,max_iter=int(1e4),tol=1e-4,learning_rate='adaptive',random_state=20140806)
self_cal_model.fit(x_train_norm,y_train[:,i])
predict_slope = self_cal_model.predict(x_train_norm)
print(f'Max of predict_slope is {predict_slope.max()}')
print(f'Min of predict_slope is {predict_slope.min()}')
# predict_slope[predict_slope<0.12] = 1
# predict_slope[predict_slope>2.4] = 1
mse_train_selfcal = np.mean((predict_slope - y_train[:,i])**2)
mse_train_today = np.mean((1-y_train[:,i])**2)
print(f'The model today has MSE={mse_train_today}')
print(f'The self cal model has MSE={mse_train_selfcal}')

predict_slope_test = self_cal_model.predict(x_test_norm)
print(f'Max of predict_slope_test is {predict_slope_test.max()}')
print(f'Min of predict_slope_test is {predict_slope_test.min()}')
# predict_slope_test[predict_slope_test<0.12] = 1
# predict_slope_test[predict_slope_test>2.4] = 1    
mse_test_selfcal = np.mean((predict_slope_test - y_test[:,i])**2)
mse_test_today = np.mean((1-y_test[:,i])**2)
print(f'The model today on test has MSE={mse_test_today}')
print(f'The self cal model on test has MSE={mse_test_selfcal}')
self_cal_model_dict[f'self_cal_model_{i}'] = self_cal_model

Optimizing parameter 2
Max of predict_slope is 1.3868113209078223
Min of predict_slope is 0.4810956949676958
The model today has MSE=0.09219948658034981
The self cal model has MSE=0.06862182328656398
Max of predict_slope_test is 1.3693525554671102
Min of predict_slope_test is 0.478753659047818
The model today on test has MSE=0.07145412868691003
The self cal model on test has MSE=0.07411799675960375


In [38]:
# For channel 3 intercept
i = 3
print(f'Optimizing parameter {i}')
self_cal_model = MLPRegressor(hidden_layer_sizes=(10,10,10,1),activation='logistic',solver='adam',alpha=1e-4,max_iter=int(1e4),tol=1e-4,learning_rate='adaptive',random_state=20140806)
self_cal_model.fit(x_train_norm,y_train[:,i])
predict_intercept = self_cal_model.predict(x_train_norm)
print(f'Max of predict_intercept is {predict_intercept.max()}')
print(f'Min of predict_intercept is {predict_intercept.min()}')    
# predict_intercept[predict_intercept<-100] = 0
# predict_intercept[predict_intercept>100] = 0   
mse_train_selfcal = np.mean((predict_intercept - y_train[:,i])**2)
mse_train_today = np.mean(y_train[:,i]**2)
print(f'The model today has MSE={mse_train_today}')
print(f'The self cal model has MSE={mse_train_selfcal}')

predict_intercept_test = self_cal_model.predict(x_test_norm)
print(f'Max of predict_intercept_test is {predict_intercept_test.max()}')
print(f'Min of predict_intercept_test is {predict_intercept_test.min()}')    
# predict_intercept_test[predict_intercept_test<-100] = 0
# predict_intercept_test[predict_intercept_test>100] = 0     
mse_test_selfcal = np.mean((predict_intercept_test - y_test[:,i])**2)
mse_test_today = np.mean(y_test[:,i]**2)
print(f'The model today on test has MSE={mse_test_today}')
print(f'The self cal model on test has MSE={mse_test_selfcal}')
self_cal_model_dict[f'self_cal_model_{i}'] = self_cal_model

Optimizing parameter 3
Max of predict_intercept is 93.38715984045159
Min of predict_intercept is -95.47482315594551
The model today has MSE=941.7469927194347
The self cal model has MSE=659.5401963199821
Max of predict_intercept_test is 81.10202032795473
Min of predict_intercept_test is -95.47482315594551
The model today on test has MSE=602.4130508484769
The self cal model on test has MSE=853.6724571988706


In [39]:
ch3_slope_train = self_cal_model_dict['self_cal_model_2'].predict(x_train_norm)
print(f'Max of predict_slope is {ch3_slope_train.max()}')
print(f'Min of predict_slope is {ch3_slope_train.min()}')
ch3_intercept_train = self_cal_model_dict['self_cal_model_3'].predict(x_train_norm)
print(f'Max of predict_intercept is {ch3_intercept_train.max()}')
print(f'Min of predict_intercept is {ch3_intercept_train.min()}')
df_training_valid['model_output_s3_selfcal'] = df_training_valid['model_output_s3_calculateGlucose'].sub(ch3_intercept_train).div(ch3_slope_train)
n_ch3_train_today = df_training_valid['model_output_s3_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).count()
mrd_ch3_train_today = df_training_valid['model_output_s3_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).mean()
stdrd_ch3_train_today = df_training_valid['model_output_s3_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).std()
mard_ch3_train_today = df_training_valid['model_output_s3_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).abs().mean()
n_ch3_train_selfcal = df_training_valid['model_output_s3_selfcal'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).count()
mrd_ch3_train_selfcal = df_training_valid['model_output_s3_selfcal'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).mean()
stdrd_ch3_train_selfcal = df_training_valid['model_output_s3_selfcal'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).std()
mard_ch3_train_selfcal = df_training_valid['model_output_s3_selfcal'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).abs().mean()
n_ch3_train_retro = df_training_valid['retro_model_output_s3_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).count()
mrd_ch3_train_retro = df_training_valid['retro_model_output_s3_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).mean()
stdrd_ch3_train_retro = df_training_valid['retro_model_output_s3_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).std()
mard_ch3_train_retro = df_training_valid['retro_model_output_s3_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).abs().mean()
print(f'Training data for ch3 glucose by model today N = {n_ch3_train_today}, MRD = {mrd_ch3_train_today}, StdRD = {stdrd_ch3_train_today}, MARD = {mard_ch3_train_today}')
print(f'Training data for ch3 glucose by selfcal N = {n_ch3_train_selfcal}, MRD = {mrd_ch3_train_selfcal}, StdRD = {stdrd_ch3_train_selfcal}, MARD = {mard_ch3_train_selfcal}')
print(f'Training data for ch3 glucose by retro N = {n_ch3_train_retro}, MRD = {mrd_ch3_train_retro}, StdRD = {stdrd_ch3_train_retro}, MARD = {mard_ch3_train_retro}')

ch3_slope_test = self_cal_model_dict['self_cal_model_2'].predict(x_test_norm)
print(f'Max of predict_slope is {ch3_slope_test.max()}')
print(f'Min of predict_slope is {ch3_slope_test.min()}')
ch3_intercept_test = self_cal_model_dict['self_cal_model_3'].predict(x_test_norm)
print(f'Max of predict_intercept is {ch3_intercept_test.max()}')
print(f'Min of predict_intercept is {ch3_intercept_test.min()}')
df_test_valid['model_output_s3_selfcal'] = df_test_valid['model_output_s3_calculateGlucose'].sub(ch3_intercept_test).div(ch3_slope_test)
n_ch3_test_today = df_test_valid['model_output_s3_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).count()
mrd_ch3_test_today = df_test_valid['model_output_s3_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).mean()
stdrd_ch3_test_today = df_test_valid['model_output_s3_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).std()
mard_ch3_test_today = df_test_valid['model_output_s3_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).abs().mean()
n_ch3_test_selfcal = df_test_valid['model_output_s3_selfcal'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).count()
mrd_ch3_test_selfcal = df_test_valid['model_output_s3_selfcal'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).mean()
stdrd_ch3_test_selfcal = df_test_valid['model_output_s3_selfcal'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).std()
mard_ch3_test_selfcal = df_test_valid['model_output_s3_selfcal'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).abs().mean()
n_ch3_test_retro = df_test_valid['retro_model_output_s3_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).count()
mrd_ch3_test_retro = df_test_valid['retro_model_output_s3_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).mean()
stdrd_ch3_test_retro = df_test_valid['retro_model_output_s3_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).std()
mard_ch3_test_retro = df_test_valid['retro_model_output_s3_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).abs().mean()
print(f'Test data for ch3 glucose by model today N = {n_ch3_test_today}, MRD = {mrd_ch3_test_today}, StdRD = {stdrd_ch3_test_today}, MARD = {mard_ch3_test_today}')
print(f'Test data for ch3 glucose by selfcal N = {n_ch3_test_selfcal}, MRD = {mrd_ch3_test_selfcal}, StdRD = {stdrd_ch3_test_selfcal}, MARD = {mard_ch3_test_selfcal}')
print(f'Test data for ch3 glucose by retro N = {n_ch3_test_retro}, MRD = {mrd_ch3_test_retro}, StdRD = {stdrd_ch3_test_retro}, MARD = {mard_ch3_test_retro}')

Max of predict_slope is 1.3868113209078223
Min of predict_slope is 0.4810956949676958
Max of predict_intercept is 93.38715984045159
Min of predict_intercept is -95.47482315594551
Training data for ch3 glucose by model today N = 6225452, MRD = -0.10880496163635991, StdRD = 0.409228767068777, MARD = 0.24512183237647348
Training data for ch3 glucose by selfcal N = 6225452, MRD = -0.008241303522683593, StdRD = 0.46287787282523996, MARD = 0.21621806844862423
Training data for ch3 glucose by retro N = 6225452, MRD = -0.0012286261266750648, StdRD = 0.44368992395898393, MARD = 0.18188229257663047
Max of predict_slope is 1.3693525554671102
Min of predict_slope is 0.478753659047818
Max of predict_intercept is 81.10202032795473
Min of predict_intercept is -95.47482315594551
Test data for ch3 glucose by model today N = 659457, MRD = -0.20476619751489103, StdRD = 0.2486731148925224, MARD = 0.2633722332208764
Test data for ch3 glucose by selfcal N = 659457, MRD = -0.010198558239254842, StdRD = 0.357

In [40]:
df = df_orig[all_cols].copy()

df_train_ysi = df.loc[training_index]
x_train_ysi = df_train_ysi[feature_col]
x_train_ysi_norm = min_max_scaler.transform(x_train_ysi)
ch3_slope_train = self_cal_model_dict['self_cal_model_2'].predict(x_train_ysi_norm)
print(f'Max of predict_slope is {ch3_slope_train.max()}')
print(f'Min of predict_slope is {ch3_slope_train.min()}')
ch3_intercept_train = self_cal_model_dict['self_cal_model_3'].predict(x_train_ysi_norm)
print(f'Max of predict_intercept is {ch3_intercept_train.max()}')
print(f'Min of predict_intercept is {ch3_intercept_train.min()}')
df_train_ysi['model_output_s3_selfcal'] = df_train_ysi['model_output_s3_calculateGlucose'].sub(ch3_intercept_train).div(ch3_slope_train)
n_ch3_train_today = df_train_ysi['model_output_s3_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).count()
mrd_ch3_train_today = df_train_ysi['model_output_s3_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).mean()
stdrd_ch3_train_today = df_train_ysi['model_output_s3_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).std()
mard_ch3_train_today = df_train_ysi['model_output_s3_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).abs().mean()
n_ch3_train_selfcal = df_train_ysi['model_output_s3_selfcal'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).count()
mrd_ch3_train_selfcal = df_train_ysi['model_output_s3_selfcal'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).mean()
stdrd_ch3_train_selfcal = df_train_ysi['model_output_s3_selfcal'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).std()
mard_ch3_train_selfcal = df_train_ysi['model_output_s3_selfcal'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).abs().mean()
n_ch3_train_retro = df_train_ysi['retro_model_output_s3_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).count()
mrd_ch3_train_retro = df_train_ysi['retro_model_output_s3_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).mean()
stdrd_ch3_train_retro = df_train_ysi['retro_model_output_s3_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).std()
mard_ch3_train_retro = df_train_ysi['retro_model_output_s3_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).abs().mean()
print(f'Training data for ch3 glucose by model today N = {n_ch3_train_today}, MRD = {mrd_ch3_train_today}, StdRD = {stdrd_ch3_train_today}, MARD = {mard_ch3_train_today}')
print(f'Training data for ch3 glucose by selfcal N = {n_ch3_train_selfcal}, MRD = {mrd_ch3_train_selfcal}, StdRD = {stdrd_ch3_train_selfcal}, MARD = {mard_ch3_train_selfcal}')
print(f'Training data for ch3 glucose by retro N = {n_ch3_train_retro}, MRD = {mrd_ch3_train_retro}, StdRD = {stdrd_ch3_train_retro}, MARD = {mard_ch3_train_retro}')

df_test_ysi = df.loc[test_index]
x_test_ysi = df_test_ysi[feature_col]
x_test_ysi_norm = min_max_scaler.transform(x_test_ysi)
ch3_slope_test = self_cal_model_dict['self_cal_model_2'].predict(x_test_ysi_norm)
print(f'Max of predict_slope is {ch3_slope_test.max()}')
print(f'Min of predict_slope is {ch3_slope_test.min()}')
ch3_intercept_test = self_cal_model_dict['self_cal_model_3'].predict(x_test_ysi_norm)
print(f'Max of predict_intercept is {ch3_intercept_test.max()}')
print(f'Min of predict_intercept is {ch3_intercept_test.min()}')
df_test_ysi['model_output_s3_selfcal'] = df_test_ysi['model_output_s3_calculateGlucose'].sub(ch3_intercept_test).div(ch3_slope_test)
n_ch3_test_today = df_test_ysi['model_output_s3_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch3_test_today = df_test_ysi['model_output_s3_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch3_test_today = df_test_ysi['model_output_s3_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch3_test_today = df_test_ysi['model_output_s3_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
n_ch3_test_selfcal = df_test_ysi['model_output_s3_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch3_test_selfcal = df_test_ysi['model_output_s3_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch3_test_selfcal = df_test_ysi['model_output_s3_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch3_test_selfcal = df_test_ysi['model_output_s3_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
n_ch3_test_retro = df_test_ysi['retro_model_output_s3_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch3_test_retro = df_test_ysi['retro_model_output_s3_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch3_test_retro = df_test_ysi['retro_model_output_s3_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch3_test_retro = df_test_ysi['retro_model_output_s3_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
print(f'Test data for ch3 glucose by model today N = {n_ch3_test_today}, MRD = {mrd_ch3_test_today}, StdRD = {stdrd_ch3_test_today}, MARD = {mard_ch3_test_today}')
print(f'Test data for ch3 glucose by selfcal N = {n_ch3_test_selfcal}, MRD = {mrd_ch3_test_selfcal}, StdRD = {stdrd_ch3_test_selfcal}, MARD = {mard_ch3_test_selfcal}')
print(f'Test data for ch3 glucose by retro N = {n_ch3_test_retro}, MRD = {mrd_ch3_test_retro}, StdRD = {stdrd_ch3_test_retro}, MARD = {mard_ch3_test_retro}')

Max of predict_slope is 1.4241396556365038
Min of predict_slope is 0.4787555149918739
Max of predict_intercept is 94.25484559333054
Min of predict_intercept is -95.47482315594551


/var/folders/nr/d5llxsx51b3fljj36q8cwm1c0000gp/T/ipykernel_8323/1433483117.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Training data for ch3 glucose by model today N = 328214, MRD = -0.08634752983348658, StdRD = 0.2898881111672664, MARD = 0.22427636514724378
Training data for ch3 glucose by selfcal N = 328214, MRD = -0.012464455119047492, StdRD = 0.28740442710512826, MARD = 0.21016223944667495
Training data for ch3 glucose by retro N = 323773, MRD = 0.05592438308176151, StdRD = 0.3074331958547224, MARD = 0.18449894316800203
Max of predict_slope is 1.3693525554671102
Min of predict_slope is 0.478753659047818
Max of predict_intercept is 81.10202032795473
Min of predict_intercept is -95.47482315594551
Test data for ch3 glucose by model today N = 44042, MRD = -0.19906504226935867, StdRD = 0.23780076772625733, MARD = 0.24578955182041193
Test data for ch3 glucose by selfcal N = 44042, MRD = 0.07657641278861624, StdRD = 0.4029151038148268, MARD = 0.27208211493983114
Test data for ch3 glucose by retro N = 44042, MRD = 0.016434481921882578, StdRD = 0.24627389531784993, MARD = 0.16627167897166684


/var/folders/nr/d5llxsx51b3fljj36q8cwm1c0000gp/T/ipykernel_8323/1433483117.py:38: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [41]:
df = df_orig[all_cols].copy()

df_test_ysi = df.loc[training_index]
x_train_ysi = df_test_ysi[feature_col]
x_train_ysi_norm = min_max_scaler.transform(x_train_ysi)

ch3_slope_train = self_cal_model_dict['self_cal_model_2'].predict(x_train_ysi_norm)
print(f'Max of predict_slope is {ch3_slope_train.max()}')
print(f'Min of predict_slope is {ch3_slope_train.min()}')
ch3_intercept_train = self_cal_model_dict['self_cal_model_3'].predict(x_train_ysi_norm)
print(f'Max of predict_intercept is {ch3_intercept_train.max()}')
print(f'Min of predict_intercept is {ch3_intercept_train.min()}')
df_test_ysi['biolinq_s3_selfcal'] = df_test_ysi['biolinq_s3_channelFaultDetection'].sub(ch3_intercept_train).div(ch3_slope_train)
n_ch3_train_today = df_test_ysi['biolinq_s3_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch3_train_today = df_test_ysi['biolinq_s3_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch3_train_today = df_test_ysi['biolinq_s3_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch3_train_today = df_test_ysi['biolinq_s3_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
n_ch3_train_selfcal = df_test_ysi['biolinq_s3_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch3_train_selfcal = df_test_ysi['biolinq_s3_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch3_train_selfcal = df_test_ysi['biolinq_s3_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch3_train_selfcal = df_test_ysi['biolinq_s3_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
n_ch3_train_retro = df_test_ysi['retro_model_output_s3_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch3_train_retro = df_test_ysi['retro_model_output_s3_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch3_train_retro = df_test_ysi['retro_model_output_s3_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch3_train_retro = df_test_ysi['retro_model_output_s3_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
print(f'Training data for ch3 glucose by model today N = {n_ch3_train_today}, MRD = {mrd_ch3_train_today}, StdRD = {stdrd_ch3_train_today}, MARD = {mard_ch3_train_today}')
print(f'Training data for ch3 glucose by selfcal N = {n_ch3_train_selfcal}, MRD = {mrd_ch3_train_selfcal}, StdRD = {stdrd_ch3_train_selfcal}, MARD = {mard_ch3_train_selfcal}')
print(f'Training data for ch3 glucose by retro N = {n_ch3_train_retro}, MRD = {mrd_ch3_train_retro}, StdRD = {stdrd_ch3_train_retro}, MARD = {mard_ch3_train_retro}')

df_test_ysi = df.loc[test_index]
x_test_ysi = df_test_ysi[feature_col]
x_test_ysi_norm = min_max_scaler.transform(x_test_ysi)
ch3_slope_test = self_cal_model_dict['self_cal_model_2'].predict(x_test_ysi_norm)
print(f'Max of predict_slope is {ch3_slope_test.max()}')
print(f'Min of predict_slope is {ch3_slope_test.min()}')
ch3_intercept_test = self_cal_model_dict['self_cal_model_3'].predict(x_test_ysi_norm)
print(f'Max of predict_intercept is {ch3_intercept_test.max()}')
print(f'Min of predict_intercept is {ch3_intercept_test.min()}')
df_test_ysi['biolinq_s3_selfcal'] = df_test_ysi['biolinq_s3_channelFaultDetection'].sub(ch3_intercept_test).div(ch3_slope_test)
n_ch3_test_today = df_test_ysi['biolinq_s3_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch3_test_today = df_test_ysi['biolinq_s3_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch3_test_today = df_test_ysi['biolinq_s3_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch3_test_today = df_test_ysi['biolinq_s3_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
n_ch3_test_selfcal = df_test_ysi['biolinq_s3_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch3_test_selfcal = df_test_ysi['biolinq_s3_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch3_test_selfcal = df_test_ysi['biolinq_s3_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch3_test_selfcal = df_test_ysi['biolinq_s3_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
n_ch3_test_retro = df_test_ysi['retro_model_output_s3_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch3_test_retro = df_test_ysi['retro_model_output_s3_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch3_test_retro = df_test_ysi['retro_model_output_s3_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch3_test_retro = df_test_ysi['retro_model_output_s3_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
print(f'Test data for ch3 glucose by model today N = {n_ch3_test_today}, MRD = {mrd_ch3_test_today}, StdRD = {stdrd_ch3_test_today}, MARD = {mard_ch3_test_today}')
print(f'Test data for ch3 glucose by selfcal N = {n_ch3_test_selfcal}, MRD = {mrd_ch3_test_selfcal}, StdRD = {stdrd_ch3_test_selfcal}, MARD = {mard_ch3_test_selfcal}')
print(f'Test data for ch3 glucose by retro N = {n_ch3_test_retro}, MRD = {mrd_ch3_test_retro}, StdRD = {stdrd_ch3_test_retro}, MARD = {mard_ch3_test_retro}')

Max of predict_slope is 1.4241396556365038
Min of predict_slope is 0.4787555149918739
Max of predict_intercept is 94.25484559333054
Min of predict_intercept is -95.47482315594551


/var/folders/nr/d5llxsx51b3fljj36q8cwm1c0000gp/T/ipykernel_8323/3716981306.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Training data for ch3 glucose by model today N = 299893, MRD = -0.034300866234357866, StdRD = 0.2351584363253289, MARD = 0.18499737199752148
Training data for ch3 glucose by selfcal N = 299893, MRD = 0.02961000200701586, StdRD = 0.24521345141145434, MARD = 0.18446864385982004
Training data for ch3 glucose by retro N = 323773, MRD = 0.05592438308176151, StdRD = 0.3074331958547224, MARD = 0.18449894316800203
Max of predict_slope is 1.3693525554671102
Min of predict_slope is 0.478753659047818
Max of predict_intercept is 81.10202032795473
Min of predict_intercept is -95.47482315594551
Test data for ch3 glucose by model today N = 39213, MRD = -0.1650333051989289, StdRD = 0.19897222631154865, MARD = 0.21350696450060477
Test data for ch3 glucose by selfcal N = 39213, MRD = 0.11673652682934277, StdRD = 0.39569336910203706, MARD = 0.26689787795937714
Test data for ch3 glucose by retro N = 44042, MRD = 0.016434481921882578, StdRD = 0.24627389531784993, MARD = 0.16627167897166684


/var/folders/nr/d5llxsx51b3fljj36q8cwm1c0000gp/T/ipykernel_8323/3716981306.py:39: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [42]:
[ch3_slope_train.min(),
ch3_slope_train.max(),
ch3_intercept_train.min(),
ch3_intercept_train.max(),
ch3_slope_test.min(),
ch3_slope_test.max(),
ch3_intercept_test.min(),
ch3_intercept_test.max()]

[0.4787555149918739,
 1.4241396556365038,
 -95.47482315594551,
 94.25484559333054,
 0.478753659047818,
 1.3693525554671102,
 -95.47482315594551,
 81.10202032795473]

In [43]:
# For channel 4 slope
i = 4
print(f'Optimizing parameter {i}')
self_cal_model = MLPRegressor(hidden_layer_sizes=(10,10,10,1),activation='logistic',solver='adam',alpha=1e-4,max_iter=int(1e4),tol=1e-4,learning_rate='adaptive',random_state=20140806)
self_cal_model.fit(x_train_norm,y_train[:,i])
predict_slope = self_cal_model.predict(x_train_norm)
print(f'Max of predict_slope is {predict_slope.max()}')
print(f'Min of predict_slope is {predict_slope.min()}')
# predict_slope[predict_slope<0.12] = 1
# predict_slope[predict_slope>2.4] = 1
mse_train_selfcal = np.mean((predict_slope - y_train[:,i])**2)
mse_train_today = np.mean((1-y_train[:,i])**2)
print(f'The model today has MSE={mse_train_today}')
print(f'The self cal model has MSE={mse_train_selfcal}')

predict_slope_test = self_cal_model.predict(x_test_norm)
print(f'Max of predict_slope_test is {predict_slope_test.max()}')
print(f'Min of predict_slope_test is {predict_slope_test.min()}')
# predict_slope_test[predict_slope_test<0.12] = 1
# predict_slope_test[predict_slope_test>2.4] = 1    
mse_test_selfcal = np.mean((predict_slope_test - y_test[:,i])**2)
mse_test_today = np.mean((1-y_test[:,i])**2)
print(f'The model today on test has MSE={mse_test_today}')
print(f'The self cal model on test has MSE={mse_test_selfcal}')
self_cal_model_dict[f'self_cal_model_{i}'] = self_cal_model

Optimizing parameter 4
Max of predict_slope is 1.3206912555003794
Min of predict_slope is 0.4185998756933438
The model today has MSE=0.1001996416443088
The self cal model has MSE=0.06784292226244772
Max of predict_slope_test is 1.3110785608485562
Min of predict_slope_test is 0.4196156468012129
The model today on test has MSE=0.07504539794682545
The self cal model on test has MSE=0.07180278207820973


In [44]:
# For channel 4 intercept
i = 5
print(f'Optimizing parameter {i}')
self_cal_model = MLPRegressor(hidden_layer_sizes=(10,10,10,1),activation='logistic',solver='adam',alpha=1e-4,max_iter=int(1e4),tol=1e-4,learning_rate='adaptive',random_state=20140806)
self_cal_model.fit(x_train_norm,y_train[:,i])
predict_intercept = self_cal_model.predict(x_train_norm)
print(f'Max of predict_intercept is {predict_intercept.max()}')
print(f'Min of predict_intercept is {predict_intercept.min()}')    
# predict_intercept[predict_intercept<-100] = 0
# predict_intercept[predict_intercept>100] = 0   
mse_train_selfcal = np.mean((predict_intercept - y_train[:,i])**2)
mse_train_today = np.mean(y_train[:,i]**2)
print(f'The model today has MSE={mse_train_today}')
print(f'The self cal model has MSE={mse_train_selfcal}')

predict_intercept_test = self_cal_model.predict(x_test_norm)
print(f'Max of predict_intercept_test is {predict_intercept_test.max()}')
print(f'Min of predict_intercept_test is {predict_intercept_test.min()}')    
# predict_intercept_test[predict_intercept_test<-100] = 0
# predict_intercept_test[predict_intercept_test>100] = 0     
mse_test_selfcal = np.mean((predict_intercept_test - y_test[:,i])**2)
mse_test_today = np.mean(y_test[:,i]**2)
print(f'The model today on test has MSE={mse_test_today}')
print(f'The self cal model on test has MSE={mse_test_selfcal}')
self_cal_model_dict[f'self_cal_model_{i}'] = self_cal_model

Optimizing parameter 5
Max of predict_intercept is 15.671346380524206
Min of predict_intercept is -105.57807970379227
The model today has MSE=821.1604027378901
The self cal model has MSE=615.8181971900332
Max of predict_intercept_test is 15.671345546971608
Min of predict_intercept_test is -105.46185347753381
The model today on test has MSE=610.4872071965033
The self cal model on test has MSE=662.2675975760822


In [45]:
ch4_slope_train = self_cal_model_dict['self_cal_model_4'].predict(x_train_norm)
print(f'Max of predict_slope is {ch4_slope_train.max()}')
print(f'Min of predict_slope is {ch4_slope_train.min()}')
ch4_intercept_train = self_cal_model_dict['self_cal_model_5'].predict(x_train_norm)
print(f'Max of predict_intercept is {ch4_intercept_train.max()}')
print(f'Min of predict_intercept is {ch4_intercept_train.min()}')
df_training_valid['model_output_s4_selfcal'] = df_training_valid['model_output_s4_calculateGlucose'].sub(ch4_intercept_train).div(ch4_slope_train)
n_ch4_train_today = df_training_valid['model_output_s4_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).count()
mrd_ch4_train_today = df_training_valid['model_output_s4_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).mean()
stdrd_ch4_train_today = df_training_valid['model_output_s4_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).std()
mard_ch4_train_today = df_training_valid['model_output_s4_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).abs().mean()
n_ch4_train_selfcal = df_training_valid['model_output_s4_selfcal'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).count()
mrd_ch4_train_selfcal = df_training_valid['model_output_s4_selfcal'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).mean()
stdrd_ch4_train_selfcal = df_training_valid['model_output_s4_selfcal'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).std()
mard_ch4_train_selfcal = df_training_valid['model_output_s4_selfcal'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).abs().mean()
n_ch4_train_retro = df_training_valid['retro_model_output_s4_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).count()
mrd_ch4_train_retro = df_training_valid['retro_model_output_s4_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).mean()
stdrd_ch4_train_retro = df_training_valid['retro_model_output_s4_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).std()
mard_ch4_train_retro = df_training_valid['retro_model_output_s4_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).abs().mean()
print(f'Training data for ch4 glucose by model today N = {n_ch4_train_today}, MRD = {mrd_ch4_train_today}, StdRD = {stdrd_ch4_train_today}, MARD = {mard_ch4_train_today}')
print(f'Training data for ch4 glucose by selfcal N = {n_ch4_train_selfcal}, MRD = {mrd_ch4_train_selfcal}, StdRD = {stdrd_ch4_train_selfcal}, MARD = {mard_ch4_train_selfcal}')
print(f'Training data for ch4 glucose by retro N = {n_ch4_train_retro}, MRD = {mrd_ch4_train_retro}, StdRD = {stdrd_ch4_train_retro}, MARD = {mard_ch4_train_retro}')

ch4_slope_test = self_cal_model_dict['self_cal_model_4'].predict(x_test_norm)
print(f'Max of predict_slope is {ch4_slope_test.max()}')
print(f'Min of predict_slope is {ch4_slope_test.min()}')
ch4_intercept_test = self_cal_model_dict['self_cal_model_5'].predict(x_test_norm)
print(f'Max of predict_intercept is {ch4_intercept_test.max()}')
print(f'Min of predict_intercept is {ch4_intercept_test.min()}')
df_test_valid['model_output_s4_selfcal'] = df_test_valid['model_output_s4_calculateGlucose'].sub(ch4_intercept_test).div(ch4_slope_test)
n_ch4_test_today = df_test_valid['model_output_s4_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).count()
mrd_ch4_test_today = df_test_valid['model_output_s4_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).mean()
stdrd_ch4_test_today = df_test_valid['model_output_s4_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).std()
mard_ch4_test_today = df_test_valid['model_output_s4_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).abs().mean()
n_ch4_test_selfcal = df_test_valid['model_output_s4_selfcal'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).count()
mrd_ch4_test_selfcal = df_test_valid['model_output_s4_selfcal'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).mean()
stdrd_ch4_test_selfcal = df_test_valid['model_output_s4_selfcal'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).std()
mard_ch4_test_selfcal = df_test_valid['model_output_s4_selfcal'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).abs().mean()
n_ch4_test_retro = df_test_valid['retro_model_output_s4_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).count()
mrd_ch4_test_retro = df_test_valid['retro_model_output_s4_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).mean()
stdrd_ch4_test_retro = df_test_valid['retro_model_output_s4_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).std()
mard_ch4_test_retro = df_test_valid['retro_model_output_s4_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).abs().mean()
print(f'Test data for ch4 glucose by model today N = {n_ch4_test_today}, MRD = {mrd_ch4_test_today}, StdRD = {stdrd_ch4_test_today}, MARD = {mard_ch4_test_today}')
print(f'Test data for ch4 glucose by selfcal N = {n_ch4_test_selfcal}, MRD = {mrd_ch4_test_selfcal}, StdRD = {stdrd_ch4_test_selfcal}, MARD = {mard_ch4_test_selfcal}')
print(f'Test data for ch4 glucose by retro N = {n_ch4_test_retro}, MRD = {mrd_ch4_test_retro}, StdRD = {stdrd_ch4_test_retro}, MARD = {mard_ch4_test_retro}')

Max of predict_slope is 1.3206912555003794
Min of predict_slope is 0.4185998756933438
Max of predict_intercept is 15.671346380524206
Min of predict_intercept is -105.57807970379227
Training data for ch4 glucose by model today N = 6225452, MRD = -0.04944649597725861, StdRD = 0.542968624268914, MARD = 0.24282285782414065
Training data for ch4 glucose by selfcal N = 6225452, MRD = -0.030022982048584997, StdRD = 0.6882994824812046, MARD = 0.21682477300829586
Training data for ch4 glucose by retro N = 6225452, MRD = 0.005126882339610347, StdRD = 0.5479741842961442, MARD = 0.18758759950567067
Max of predict_slope is 1.3110785608485562
Min of predict_slope is 0.4196156468012129
Max of predict_intercept is 15.671345546971608
Min of predict_intercept is -105.46185347753381
Test data for ch4 glucose by model today N = 659457, MRD = -0.1784356522690694, StdRD = 0.2636117484456087, MARD = 0.25253750679896997
Test data for ch4 glucose by selfcal N = 659457, MRD = -0.025539777738035938, StdRD = 0.36

In [46]:
df = df_orig[all_cols].copy()

df_train_ysi = df.loc[training_index]
x_train_ysi = df_train_ysi[feature_col]
x_train_ysi_norm = min_max_scaler.transform(x_train_ysi)
ch4_slope_train = self_cal_model_dict['self_cal_model_4'].predict(x_train_ysi_norm)
print(f'Max of predict_slope is {ch4_slope_train.max()}')
print(f'Min of predict_slope is {ch4_slope_train.min()}')
ch4_intercept_train = self_cal_model_dict['self_cal_model_5'].predict(x_train_ysi_norm)
print(f'Max of predict_intercept is {ch4_intercept_train.max()}')
print(f'Min of predict_intercept is {ch4_intercept_train.min()}')
df_train_ysi['model_output_s4_selfcal'] = df_train_ysi['model_output_s4_calculateGlucose'].sub(ch4_intercept_train).div(ch4_slope_train)
n_ch4_train_today = df_train_ysi['model_output_s4_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).count()
mrd_ch4_train_today = df_train_ysi['model_output_s4_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).mean()
stdrd_ch4_train_today = df_train_ysi['model_output_s4_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).std()
mard_ch4_train_today = df_train_ysi['model_output_s4_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).abs().mean()
n_ch4_train_selfcal = df_train_ysi['model_output_s4_selfcal'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).count()
mrd_ch4_train_selfcal = df_train_ysi['model_output_s4_selfcal'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).mean()
stdrd_ch4_train_selfcal = df_train_ysi['model_output_s4_selfcal'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).std()
mard_ch4_train_selfcal = df_train_ysi['model_output_s4_selfcal'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).abs().mean()
n_ch4_train_retro = df_train_ysi['retro_model_output_s4_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).count()
mrd_ch4_train_retro = df_train_ysi['retro_model_output_s4_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).mean()
stdrd_ch4_train_retro = df_train_ysi['retro_model_output_s4_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).std()
mard_ch4_train_retro = df_train_ysi['retro_model_output_s4_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).abs().mean()
print(f'Training data for ch4 glucose by model today N = {n_ch4_train_today}, MRD = {mrd_ch4_train_today}, StdRD = {stdrd_ch4_train_today}, MARD = {mard_ch4_train_today}')
print(f'Training data for ch4 glucose by selfcal N = {n_ch4_train_selfcal}, MRD = {mrd_ch4_train_selfcal}, StdRD = {stdrd_ch4_train_selfcal}, MARD = {mard_ch4_train_selfcal}')
print(f'Training data for ch4 glucose by retro N = {n_ch4_train_retro}, MRD = {mrd_ch4_train_retro}, StdRD = {stdrd_ch4_train_retro}, MARD = {mard_ch4_train_retro}')

df_test_ysi = df.loc[test_index]
x_test_ysi = df_test_ysi[feature_col]
x_test_ysi_norm = min_max_scaler.transform(x_test_ysi)
ch4_slope_test = self_cal_model_dict['self_cal_model_4'].predict(x_test_ysi_norm)
print(f'Max of predict_slope is {ch4_slope_test.max()}')
print(f'Min of predict_slope is {ch4_slope_test.min()}')
ch4_intercept_test = self_cal_model_dict['self_cal_model_5'].predict(x_test_ysi_norm)
print(f'Max of predict_intercept is {ch4_intercept_test.max()}')
print(f'Min of predict_intercept is {ch4_intercept_test.min()}')
df_test_ysi['model_output_s4_selfcal'] = df_test_ysi['model_output_s4_calculateGlucose'].sub(ch4_intercept_test).div(ch4_slope_test)
n_ch4_test_today = df_test_ysi['model_output_s4_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch4_test_today = df_test_ysi['model_output_s4_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch4_test_today = df_test_ysi['model_output_s4_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch4_test_today = df_test_ysi['model_output_s4_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
n_ch4_test_selfcal = df_test_ysi['model_output_s4_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch4_test_selfcal = df_test_ysi['model_output_s4_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch4_test_selfcal = df_test_ysi['model_output_s4_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch4_test_selfcal = df_test_ysi['model_output_s4_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
n_ch4_test_retro = df_test_ysi['retro_model_output_s4_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch4_test_retro = df_test_ysi['retro_model_output_s4_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch4_test_retro = df_test_ysi['retro_model_output_s4_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch4_test_retro = df_test_ysi['retro_model_output_s4_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
print(f'Test data for ch4 glucose by model today N = {n_ch4_test_today}, MRD = {mrd_ch4_test_today}, StdRD = {stdrd_ch4_test_today}, MARD = {mard_ch4_test_today}')
print(f'Test data for ch4 glucose by selfcal N = {n_ch4_test_selfcal}, MRD = {mrd_ch4_test_selfcal}, StdRD = {stdrd_ch4_test_selfcal}, MARD = {mard_ch4_test_selfcal}')
print(f'Test data for ch4 glucose by retro N = {n_ch4_test_retro}, MRD = {mrd_ch4_test_retro}, StdRD = {stdrd_ch4_test_retro}, MARD = {mard_ch4_test_retro}')

Max of predict_slope is 1.3206917749119929
Min of predict_slope is 0.4185454904682045
Max of predict_intercept is 15.671346380524206
Min of predict_intercept is -105.58616422467415


/var/folders/nr/d5llxsx51b3fljj36q8cwm1c0000gp/T/ipykernel_8323/2843018395.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Training data for ch4 glucose by model today N = 328214, MRD = -0.023749111080523022, StdRD = 0.5918295056903586, MARD = 0.23344543267390838
Training data for ch4 glucose by selfcal N = 328214, MRD = -0.016837524086888774, StdRD = 0.489054473851545, MARD = 0.20776323144956507
Training data for ch4 glucose by retro N = 323773, MRD = 0.08253275813600343, StdRD = 1.209175012659502, MARD = 0.22077312802508398
Max of predict_slope is 1.3110785608485562
Min of predict_slope is 0.41817119644225653
Max of predict_intercept is 15.671346229732828
Min of predict_intercept is -105.46185347753381
Test data for ch4 glucose by model today N = 44042, MRD = -0.14730667663510766, StdRD = 0.2711408315224174, MARD = 0.23330688104352823
Test data for ch4 glucose by selfcal N = 44042, MRD = 0.09593279834457778, StdRD = 0.468924884211197, MARD = 0.3201045765420208
Test data for ch4 glucose by retro N = 44042, MRD = 0.025645503639347048, StdRD = 0.2670910317455539, MARD = 0.1793310210308623


/var/folders/nr/d5llxsx51b3fljj36q8cwm1c0000gp/T/ipykernel_8323/2843018395.py:38: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [47]:
df = df_orig[all_cols].copy()

df_test_ysi = df.loc[training_index]
x_train_ysi = df_test_ysi[feature_col]
x_train_ysi_norm = min_max_scaler.transform(x_train_ysi)

ch4_slope_train = self_cal_model_dict['self_cal_model_4'].predict(x_train_ysi_norm)
print(f'Max of predict_slope is {ch4_slope_train.max()}')
print(f'Min of predict_slope is {ch4_slope_train.min()}')
ch4_intercept_train = self_cal_model_dict['self_cal_model_5'].predict(x_train_ysi_norm)
print(f'Max of predict_intercept is {ch4_intercept_train.max()}')
print(f'Min of predict_intercept is {ch4_intercept_train.min()}')
df_test_ysi['biolinq_s4_selfcal'] = df_test_ysi['biolinq_s4_channelFaultDetection'].sub(ch4_intercept_train).div(ch4_slope_train)
n_ch4_train_today = df_test_ysi['biolinq_s4_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch4_train_today = df_test_ysi['biolinq_s4_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch4_train_today = df_test_ysi['biolinq_s4_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch4_train_today = df_test_ysi['biolinq_s4_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
n_ch4_train_selfcal = df_test_ysi['biolinq_s4_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch4_train_selfcal = df_test_ysi['biolinq_s4_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch4_train_selfcal = df_test_ysi['biolinq_s4_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch4_train_selfcal = df_test_ysi['biolinq_s4_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
n_ch4_train_retro = df_test_ysi['retro_model_output_s4_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch4_train_retro = df_test_ysi['retro_model_output_s4_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch4_train_retro = df_test_ysi['retro_model_output_s4_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch4_train_retro = df_test_ysi['retro_model_output_s4_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
print(f'Training data for ch4 glucose by model today N = {n_ch4_train_today}, MRD = {mrd_ch4_train_today}, StdRD = {stdrd_ch4_train_today}, MARD = {mard_ch4_train_today}')
print(f'Training data for ch4 glucose by selfcal N = {n_ch4_train_selfcal}, MRD = {mrd_ch4_train_selfcal}, StdRD = {stdrd_ch4_train_selfcal}, MARD = {mard_ch4_train_selfcal}')
print(f'Training data for ch4 glucose by retro N = {n_ch4_train_retro}, MRD = {mrd_ch4_train_retro}, StdRD = {stdrd_ch4_train_retro}, MARD = {mard_ch4_train_retro}')

df_test_ysi = df.loc[test_index]
x_test_ysi = df_test_ysi[feature_col]
x_test_ysi_norm = min_max_scaler.transform(x_test_ysi)
ch4_slope_test = self_cal_model_dict['self_cal_model_4'].predict(x_test_ysi_norm)
print(f'Max of predict_slope is {ch4_slope_test.max()}')
print(f'Min of predict_slope is {ch4_slope_test.min()}')
ch4_intercept_test = self_cal_model_dict['self_cal_model_5'].predict(x_test_ysi_norm)
print(f'Max of predict_intercept is {ch4_intercept_test.max()}')
print(f'Min of predict_intercept is {ch4_intercept_test.min()}')
df_test_ysi['biolinq_s4_selfcal'] = df_test_ysi['biolinq_s4_channelFaultDetection'].sub(ch4_intercept_test).div(ch4_slope_test)
n_ch4_test_today = df_test_ysi['biolinq_s4_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch4_test_today = df_test_ysi['biolinq_s4_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch4_test_today = df_test_ysi['biolinq_s4_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch4_test_today = df_test_ysi['biolinq_s4_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
n_ch4_test_selfcal = df_test_ysi['biolinq_s4_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch4_test_selfcal = df_test_ysi['biolinq_s4_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch4_test_selfcal = df_test_ysi['biolinq_s4_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch4_test_selfcal = df_test_ysi['biolinq_s4_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
n_ch4_test_retro = df_test_ysi['retro_model_output_s4_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch4_test_retro = df_test_ysi['retro_model_output_s4_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch4_test_retro = df_test_ysi['retro_model_output_s4_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch4_test_retro = df_test_ysi['retro_model_output_s4_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
print(f'Test data for ch4 glucose by model today N = {n_ch4_test_today}, MRD = {mrd_ch4_test_today}, StdRD = {stdrd_ch4_test_today}, MARD = {mard_ch4_test_today}')
print(f'Test data for ch4 glucose by selfcal N = {n_ch4_test_selfcal}, MRD = {mrd_ch4_test_selfcal}, StdRD = {stdrd_ch4_test_selfcal}, MARD = {mard_ch4_test_selfcal}')
print(f'Test data for ch4 glucose by retro N = {n_ch4_test_retro}, MRD = {mrd_ch4_test_retro}, StdRD = {stdrd_ch4_test_retro}, MARD = {mard_ch4_test_retro}')

Max of predict_slope is 1.3206917749119929
Min of predict_slope is 0.4185454904682045
Max of predict_intercept is 15.671346380524206
Min of predict_intercept is -105.58616422467415


/var/folders/nr/d5llxsx51b3fljj36q8cwm1c0000gp/T/ipykernel_8323/789112783.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Training data for ch4 glucose by model today N = 284557, MRD = -0.011362512678645383, StdRD = 0.22036229275297012, MARD = 0.16928993938617712
Training data for ch4 glucose by selfcal N = 284557, MRD = -0.004578904151191749, StdRD = 0.21690712848647664, MARD = 0.16432444878189154
Training data for ch4 glucose by retro N = 323773, MRD = 0.08253275813600343, StdRD = 1.209175012659502, MARD = 0.22077312802508398
Max of predict_slope is 1.3110785608485562
Min of predict_slope is 0.41817119644225653
Max of predict_intercept is 15.671346229732828
Min of predict_intercept is -105.46185347753381
Test data for ch4 glucose by model today N = 39728, MRD = -0.10719177651440225, StdRD = 0.22744060293808482, MARD = 0.2002691122812465
Test data for ch4 glucose by selfcal N = 39728, MRD = 0.1427433840043713, StdRD = 0.4517820312681696, MARD = 0.3065223867345288
Test data for ch4 glucose by retro N = 44042, MRD = 0.025645503639347048, StdRD = 0.2670910317455539, MARD = 0.1793310210308623


/var/folders/nr/d5llxsx51b3fljj36q8cwm1c0000gp/T/ipykernel_8323/789112783.py:39: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [48]:
[ch4_slope_train.min(),
ch4_slope_train.max(),
ch4_intercept_train.min(),
ch4_intercept_train.max(),
ch4_slope_test.min(),
ch4_slope_test.max(),
ch4_intercept_test.min(),
ch4_intercept_test.max()]

[0.4185454904682045,
 1.3206917749119929,
 -105.58616422467415,
 15.671346380524206,
 0.41817119644225653,
 1.3110785608485562,
 -105.46185347753381,
 15.671346229732828]

In [49]:
df_test_valid[['model_output_s1_selfcal','model_output_s3_selfcal','model_output_s4_selfcal']].std(1).quantile(q=0.95)

102.61180513150357

In [50]:
df_test_valid[['model_output_s1_calculateGlucose','model_output_s3_calculateGlucose','model_output_s4_calculateGlucose']].std(1).quantile(q=0.95)

51.95694160258942

In [51]:
df_test_valid[['retro_model_output_s1_calculateGlucose','retro_model_output_s3_calculateGlucose','retro_model_output_s4_calculateGlucose']].std(1).quantile(q=0.95)

44.245584844522554

In [52]:
min_max_scaler.data_min_

array([ 0.0830003, 11.9864   ,  9.79233  , 14.488    ,  2.09675  ,
        1.75432  ,  1.73353  ])

In [53]:
min_max_scaler.feature_names_in_ = np.array(['time_from_start',
                    'ewm_average_8hr_iir_filt_model_output_temp_s1',
                    'ewm_average_8hr_iir_filt_model_output_temp_s3',
                    'ewm_average_8hr_iir_filt_model_output_temp_s4',
                    'estimate_standard_deviation_8hr_iir_filt_model_output_temp_s1',
                    'estimate_standard_deviation_8hr_iir_filt_model_output_temp_s3',
                    'estimate_standard_deviation_8hr_iir_filt_model_output_temp_s4'
                    ],dtype='object')
with open (r'/Users/liangwang/GitHub/DataAnalysis/AlgorithmAssembler/Modules/calculateGlucose/selfcal_nn_models/min_max_scaler.p','wb') as fp:
    pickle.dump(min_max_scaler,fp) 

In [54]:
with open (r'/Users/liangwang/GitHub/DataAnalysis/AlgorithmAssembler/Modules/calculateGlucose/selfcal_nn_models/ch1_slope.p','wb') as fp:
    pickle.dump(self_cal_model_dict['self_cal_model_0'],fp)  
with open (r'/Users/liangwang/GitHub/DataAnalysis/AlgorithmAssembler/Modules/calculateGlucose/selfcal_nn_models/ch1_intercept.p','wb') as fp:
    pickle.dump(self_cal_model_dict['self_cal_model_1'],fp)  
with open (r'/Users/liangwang/GitHub/DataAnalysis/AlgorithmAssembler/Modules/calculateGlucose/selfcal_nn_models/ch3_slope.p','wb') as fp:
    pickle.dump(self_cal_model_dict['self_cal_model_2'],fp)  
with open (r'/Users/liangwang/GitHub/DataAnalysis/AlgorithmAssembler/Modules/calculateGlucose/selfcal_nn_models/ch3_intercept.p','wb') as fp:
    pickle.dump(self_cal_model_dict['self_cal_model_3'],fp)  
with open (r'/Users/liangwang/GitHub/DataAnalysis/AlgorithmAssembler/Modules/calculateGlucose/selfcal_nn_models/ch4_slope.p','wb') as fp:
    pickle.dump(self_cal_model_dict['self_cal_model_4'],fp)  
with open (r'/Users/liangwang/GitHub/DataAnalysis/AlgorithmAssembler/Modules/calculateGlucose/selfcal_nn_models/ch4_intercept.p','wb') as fp:
    pickle.dump(self_cal_model_dict['self_cal_model_5'],fp)                  

In [ ]:
self_cal_model_temp = MLPRegressor(solver='sgd',alpha=1e-1,max_iter=int(1e4),tol=1e-6,learning_rate='adaptive',random_state=20240528,verbose=True)
param_list = {"hidden_layer_sizes": [(10,),(10,10),(10,5,2),(2,5,10),(4,3,4),(3,2,3),(5,5),(5,)]}
gridCV = GridSearchCV(estimator=self_cal_model_temp, param_grid=param_list,cv=10,verbose=True)
gridCV.fit(x_train_norm,y_train[:,i])

In [ ]:
# with open (r'/Users/liangwang/GitHub/DataAnalysis/AlgorithmAssembler/Modules/calculateGlucose/selfcal_nn_models/min_max_scaler.p','rb') as fp:
#     min_max_scaler = pickle.load(fp)
# min_max_scaler.feature_names_in_ = np.array(['time_from_start_featureGeneration',
#        'ewm_average_8hr_iir_filt_model_output_s1_glucoseFeatureGeneration',
#        'ewm_average_8hr_iir_filt_model_output_s3_glucoseFeatureGeneration',
#        'ewm_average_8hr_iir_filt_model_output_s4_glucoseFeatureGeneration',
#        'estimate_standard_deviation_8hr_iir_filt_model_output_s1_glucoseFeatureGeneration',
#        'estimate_standard_deviation_8hr_iir_filt_model_output_s3_glucoseFeatureGeneration',
#        'estimate_standard_deviation_8hr_iir_filt_model_output_s4_glucoseFeatureGeneration'],
#       dtype='object')

In [ ]:
# self_cal_model_dict = {}
# with open (r'/Users/liangwang/GitHub/DataAnalysis/AlgorithmAssembler/Modules/calculateGlucose/selfcal_nn_models/ch1_slope.p','rb') as fp:
#     self_cal_model_dict['self_cal_model_0'] = pickle.load(fp)
# with open (r'/Users/liangwang/GitHub/DataAnalysis/AlgorithmAssembler/Modules/calculateGlucose/selfcal_nn_models/ch1_intercept.p','rb') as fp:
#     self_cal_model_dict['self_cal_model_1'] = pickle.load(fp)
# with open (r'/Users/liangwang/GitHub/DataAnalysis/AlgorithmAssembler/Modules/calculateGlucose/selfcal_nn_models/ch3_slope.p','rb') as fp:
#     self_cal_model_dict['self_cal_model_2'] = pickle.load(fp)
# with open (r'/Users/liangwang/GitHub/DataAnalysis/AlgorithmAssembler/Modules/calculateGlucose/selfcal_nn_models/ch3_intercept.p','rb') as fp:
#     self_cal_model_dict['self_cal_model_3'] = pickle.load(fp)
# with open (r'/Users/liangwang/GitHub/DataAnalysis/AlgorithmAssembler/Modules/calculateGlucose/selfcal_nn_models/ch4_slope.p','rb') as fp:
#     self_cal_model_dict['self_cal_model_4'] = pickle.load(fp)
# with open (r'/Users/liangwang/GitHub/DataAnalysis/AlgorithmAssembler/Modules/calculateGlucose/selfcal_nn_models/ch4_intercept.p','rb') as fp:
#     self_cal_model_dict['self_cal_model_5'] = pickle.load(fp)                

In [ ]:
# for i in range(6): # [0,1]:
#     print(f'Loading parameter {i}')
#     self_cal_model = self_cal_model_dict[f'self_cal_model_{i}']
#     predict_slope_intercept = self_cal_model.predict(x_train_norm)
#     mse_train_selfcal = np.mean((predict_slope_intercept - y_train[:,i])**2)
#     mse_train_today = np.mean(y_train[:,i]**2)
#     print(f'The model today has MSE={mse_train_today}')
#     print(f'The self cal model has MSE={mse_train_selfcal}')

#     predict_slope_intercept_test = self_cal_model.predict(x_test_norm)
#     mse_test_selfcal = np.mean((predict_slope_intercept_test - y_test[:,i])**2)
#     mse_test_today = np.mean(y_test[:,i]**2)
#     print(f'The model today on test has MSE={mse_test_today}')
#     print(f'The self cal model on test has MSE={mse_test_selfcal}')
#     self_cal_model_dict[f'self_cal_model_{i}'] = self_cal_model

In [ ]:
pd.DataFrame(gridCV.cv_results_)

In [ ]:
gridCV.best_estimator_

In [ ]:
i=1
self_cal_model = MLPRegressor(hidden_layer_sizes=(1,),solver='sgd',alpha=0.1,max_iter=int(1e4),tol=1e-6,learning_rate='adaptive',random_state=20240528)
self_cal_model.fit(x_train_norm,y_train[:,i])
predict_slope_intercept = self_cal_model.predict(x_train_norm)
mse_train_selfcal = np.mean((predict_slope_intercept - y_train[:,0])**2)
mse_train_today = np.mean(y_train[:,i]**2)
print(f'The model today has MSE={mse_train_today}')
print(f'The self cal model has MSE={mse_train_selfcal}')

predict_slope_intercept_test = self_cal_model.predict(x_test_norm)
mse_test_selfcal = np.mean((predict_slope_intercept_test - y_test[:,i])**2)
mse_test_today = np.mean(y_test[:,i]**2)
print(f'The model today on test has MSE={mse_test_today}')
print(f'The self cal model on test has MSE={mse_test_selfcal}')
self_cal_model_dict[f'self_cal_model_{i}'] = self_cal_model

In [ ]:
y_train[:,i].mean()

In [ ]:
df_training_valid[feature_col].plot(y='ewm_average_8hr_iir_filt_model_output_s3_glucoseFeatureGeneration',kind='hist',bins=1000)

In [ ]:
min_max_scaler.data_max_

In [ ]:
self_cal_model = MLPRegressor(hidden_layer_sizes=(10,10,10),solver='sgd',alpha=1e-4,max_iter=int(1e4),tol=1e-6,learning_rate='adaptive',random_state=20240528)
self_cal_model.fit(x_train_norm,y_train[:,0])
predict_slope_intercept = self_cal_model.predict(x_train_norm)

In [ ]:
valid_test_index = df[df['Study'].isin(test_group) & df['time_from_start_featureGeneration'].between(training_time_window[0],training_time_window[1]) & 
                        df[f'iso_s{ch_str}'].le(iso_thresh) & df[f'iso_s3'].le(iso_thresh) & df[f'iso_s4'].le(iso_thresh)].index
df_test_valid = df.loc[valid_test_index,col_for_optim_no_ysi_acck].dropna().reset_index(drop=True)
print(f'Test data has shape {df_test_valid.shape}')
x_test = df_test_valid[feature_col]
x_test_norm = min_max_scaler.fit_transform(x_test)
y_test = df_test_valid[ground_truth_col].to_numpy()

predict_slope_intercept_test = self_cal_model.predict(x_test_norm)